In [1]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

folder_path = "/kaggle/input/ai-vs-human-generated-dataset/train_data"

"""for root, dirs, files in os.walk(folder_path):
    for file in files:
        print(os.path.join(root, file))"""  # Dosyanın tam yolunu yazdır

'for root, dirs, files in os.walk(folder_path):\n    for file in files:\n        print(os.path.join(root, file))'

In [2]:
from datasets import load_dataset
from transformers import CLIPProcessor, CLIPModel
from torchvision.transforms import Compose, Resize, ToTensor, Normalize, RandomHorizontalFlip, RandomRotation, RandomResizedCrop
from torch.utils.data import DataLoader, random_split
import torch
from tqdm import tqdm
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import os
from accelerate import Accelerator

accelerator = Accelerator()

In [3]:
csv_file = "/kaggle/input/ai-vs-human-generated-dataset/train.csv"
data = pd.read_csv(csv_file)
data

,Unnamed: 0,file_name,label
0,0,train_data/a6dcb93f596a43249135678dfcfc17ea.jpg,1
1,1,train_data/041be3153810433ab146bc97d5af505c.jpg,0
2,2,train_data/615df26ce9494e5db2f70e57ce7a3a4f.jpg,1
3,3,train_data/8542fe161d9147be8e835e50c0de39cd.jpg,0
4,4,train_data/5d81fa12bc3b4cea8c94a6700a477cf2.jpg,1
...,...,...,...
79945,79945,train_data/9283b107f6274279b6f15bbe77c523aa.jpg,0
79946,79946,train_data/4c6b17fe6dd743428a45773135a10508.jpg,1
79947,79947,train_data/1ccbf96d04e342fd9f629ad55466b29e.jpg,0
79948,79948,train_data/ff960b55f296445abb3c5f304b52e104.jpg,1


In [4]:
"""import os
import shutil
import pandas as pd

# Dosyaların bulunduğu ana klasör
source_folder = "/kaggle/input/ai-vs-human-generated-dataset/"

# Hedef ana klasör
destination_root = "/kaggle/working/sorted"

# CSV dosyasının yolu
csv_file = "/kaggle/input/ai-vs-human-generated-dataset/train.csv"

# Klasörleri oluştur ve dosyaları kopyala
def organize_files():
    # CSV dosyasını pandas ile oku
    data = pd.read_csv(csv_file)

    for _, row in data.iterrows():
        file_name = row['file_name']
        label = str(row["label"])
        source_path = os.path.join(source_folder, file_name)
        destination_folder = os.path.join(destination_root, label)

        # Hedef klasörü oluştur
        os.makedirs(destination_folder, exist_ok=True)
        
        # Dosyayı kopyala
        if os.path.exists(source_path):
            shutil.copy(source_path, destination_folder)
            print(f"{file_name} kopyalandı -> {label}")
        else:
            print(f"Hata: {file_name} bulunamadı.")

organize_files()"""

'import os\nimport shutil\nimport pandas as pd\n\n# Dosyaların bulunduğu ana klasör\nsource_folder = "/kaggle/input/ai-vs-human-generated-dataset/"\n\n# Hedef ana klasör\ndestination_root = "/kaggle/working/sorted"\n\n# CSV dosyasının yolu\ncsv_file = "/kaggle/input/ai-vs-human-generated-dataset/train.csv"\n\n# Klasörleri oluştur ve dosyaları kopyala\ndef organize_files():\n    # CSV dosyasını pandas ile oku\n    data = pd.read_csv(csv_file)\n\n    for _, row in data.iterrows():\n        file_name = row[\'file_name\']\n        label = str(row["label"])\n        source_path = os.path.join(source_folder, file_name)\n        destination_folder = os.path.join(destination_root, label)\n\n        # Hedef klasörü oluştur\n        os.makedirs(destination_folder, exist_ok=True)\n        \n        # Dosyayı kopyala\n        if os.path.exists(source_path):\n            shutil.copy(source_path, destination_folder)\n            print(f"{file_name} kopyalandı -> {label}")\n        else:\n           

In [5]:
from torch.utils.data import Dataset
class MultiLabelDataset(Dataset):
  def __init__(self, root, df, transform,is_train=True):
    self.root = root
    self.df = df
    self.transform = transform
    self.is_train=is_train
  def __getitem__(self, idx):
    item = self.df.iloc[idx]
    # get image
    image_path = os.path.join(self.root, item["file_name"])
    image = Image.open(image_path).convert("RGB")

    # prepare image for the model
    pixel_values = self.transform(image)

    # get labels
    label = torch.tensor(self.df.iloc[idx]['label'])
   
    # turn into PyTorch te
    return pixel_values, label
        # For inference, return only the image
    
  def __len__(self):
    return len(self.df)

In [6]:
train_csv_path = "/kaggle/input/ai-vs-human-generated-dataset/train.csv"
train_df = pd.read_csv(train_csv_path)

# Define the base directory where images are stored
train_data_dir = "/kaggle/input/ai-vs-human-generated-dataset/"


In [7]:
labels=np.unique(train_df['label'].astype("string"))
id2label = {k: v for k, v in enumerate(labels)}
label2id = {v: k for k, v in enumerate(labels)}
print(id2label)

{0: '0', 1: '1'}


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14-336")
normalize = Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [10]:
SEED=42
train_transform = Compose([
    Resize((336, 336)),
    RandomHorizontalFlip(),
    
    
    ToTensor(),
    normalize
])

train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=SEED, stratify=train_df['label'])

# Dataset and DataLoader
train_dataset = MultiLabelDataset(train_data_dir,train_df,  transform=train_transform, is_train=True)
val_dataset = MultiLabelDataset( train_data_dir,val_df, transform=train_transform, is_train=True)

In [11]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    data = torch.stack([item[0] for item in batch])
    target = torch.stack([item[1] for item in batch])
    return data, target

train_dataloader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=8, shuffle=True)
valid_dataloader = DataLoader(val_dataset, collate_fn=collate_fn, batch_size=8)

In [12]:
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14-336")
model.config.id2label = id2label
model.config.label2id = label2id

config.json:   0%|          | 0.00/4.76k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [13]:
visual_encoder = model.vision_model

# Use DataParallel for multi-GPU training
device = "cuda" if torch.cuda.is_available() else "cpu"
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    visual_encoder = torch.nn.DataParallel(visual_encoder)

visual_encoder.to(device)


CLIPVisionTransformer(
  (embeddings): CLIPVisionEmbeddings(
    (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (position_embedding): Embedding(577, 1024)
  )
  (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (encoder): CLIPEncoder(
    (layers): ModuleList(
      (0-23): 24 x CLIPEncoderLayer(
        (self_attn): CLIPSdpaAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): CLIPMLP(
          (activation_fn): QuickGELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, 

In [14]:
from torch.optim.lr_scheduler import StepLR
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.AdamW(visual_encoder.parameters(), lr=0.00001)
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

train_losses = []
val_losses = []
accumulation_steps = 4

In [15]:
def predict_test(model, test_csv_path, test_images_dir, id2label, device, output_csv):
    test_data = pd.read_csv(test_csv_path)
    test_data["label"] = "NaN"

    model.eval()
    with torch.no_grad():
        for idx, row in tqdm(test_data.iterrows(), total=len(test_data)):
            image_path = os.path.join(test_images_dir, row["file_name"])
            if not os.path.exists(image_path):
                continue

            # Preprocess the image
            image = Image.open(image_path).convert("RGB")
            image_tensor = transform(image).unsqueeze(0).to(device)

            # Predict
            outputs = model(pixel_values=image_tensor)
            features = outputs.pooler_output
            predicted_class = torch.argmax(features, dim=-1).item()
            city_name = id2label[predicted_class]
            test_data.at[idx, "label"] = city_name

    # Save predictions
    test_data.to_csv(output_csv, index=False)
    print(f"Test predictions saved to {output_csv}")

In [16]:
for epoch in range(4):
    print("Epoch:", epoch)
    visual_encoder.train()
    running_loss = 0.0
    correct = 0
    total = 0

    optimizer.zero_grad()
    for idx, (images,labels) in enumerate(tqdm(train_dataloader)):
        images, labels = images.to(device), labels.to(device)
        
        outputs = visual_encoder(images)
        logits = outputs.pooler_output  # Extract pooled output for classification
        loss = criterion(logits,labels) / accumulation_steps
        accelerator.backward(loss)
        
        
        if (idx + 1) % accumulation_steps == 0 or (idx + 1) == len(train_dataloader):
            optimizer.step()
            optimizer.zero_grad()

        running_loss += loss.item() * accumulation_steps
        total += labels.shape[0]
        predicted = logits.argmax(-1)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_dataloader)
    train_losses.append(train_loss)
    train_accuracy = correct / total
    print(f"Train Loss: {train_loss}, Train Accuracy: {train_accuracy}")

    # Validation
    visual_encoder.eval()
    val_running_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_batch in val_dataloader:
            val_batch = {k: v.to(device) for k, v in val_batch.items()}
            val_outputs = visual_encoder(pixel_values=val_batch["pixel_values"])
            val_logits = val_outputs.pooler_output
            val_loss = criterion(val_logits, val_batch["labels"])
            val_running_loss += val_loss.item()
            val_total += val_batch["labels"].shape[0]
            val_predicted = val_logits.argmax(-1)
            val_correct += (val_predicted == val_batch["labels"]).sum().item()

    val_loss = val_running_loss / len(val_dataloader)
    val_losses.append(val_loss)
    val_accuracy = val_correct / val_total
    print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

    scheduler.step(val_loss)

    # Save the model
    model_path = f"clip_visual_model_epoch_{epoch}.pth"
    torch.save(visual_encoder.state_dict(), model_path)
    print(f"Model saved to {model_path}")

    # Test predictions
    test_csv_path = "/kaggle/input/ai-vs-human-generated-dataset/test.csv"
    test_images_dir = "/kaggle/input/ai-vs-human-generated-dataset/test_data_v2"
    submission_path = f"submission_epoch_{epoch}.csv"
    predict_test(visual_encoder, test_csv_path, test_images_dir, id2label, device, submission_path)


Epoch: 0


  1%|          | 52/7995 [01:34<4:00:28,  1.82s/it]


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/ai-vs-human-generated-dataset/train_data/f41820666d1847d791bd2da3326d2838.jpg'